In [2]:
import numpy as np
import torch
import pdb
import torch.nn as nn
import random
import matplotlib.pyplot as plt
import struct
import gzip
from PIL import Image

def load_data_image():
    #读取文件
    binfile = open('../data/train-images.idx3-ubyte', 'rb')
    buffers = binfile.read()
    #/读取前四个无符号整数
    magic, num, cows, cols = struct.unpack_from('>IIII',buffers,0)
    #获得偏移
    offset = struct.calcsize('>IIII')
    #计算所读取的数据大小num*cows*cols个bit，>表示大端读取
    strString = '>' + str(num*cows*cols) + 'B'
    #读取图片数据并转化矩阵
    training_image = struct.unpack_from(strString, buffers, offset)
    training_image = np.reshape(training_image,(num,cows*cols))
    binfile.close()
    return (training_image)
def load_data_lable():
    #读取文件
    binfile = open('../data/train-labels.idx1-ubyte', 'rb')
    buffers = binfile.read()
    #/读取前两个无符号整数
    magic, num = struct.unpack_from('>II',buffers,0)
    #获得偏移
    offset = struct.calcsize('>II')
    #计算所读取的数据大小num*cows*cols个bit，>表示大端读取
    strString = '>' + str(num) + 'B'
    #读取图片数据并转化矩阵
    training_lable = struct.unpack_from(strString, buffers, offset)
    training_lable = np.reshape(training_lable,(num))
    binfile.close()
    return (training_lable)
def vectorized_result(j):
    e = np.zeros((10, 1)) 
    e[j] = 1.0 
    return e
training_image = load_data_image()
training_lable = load_data_lable()

test_image = training_image[50000:]
test_lable = training_lable[50000:]
training_image = [np.reshape(x, (784, 1)) for x in training_image]
training_lable = [vectorized_result(y) for y in training_lable]
test_image = [np.reshape(x, (784, 1)) for x in test_image]
training_image = training_image[:50000]
training_lable = training_lable[:50000]
training_data = zip(training_image, training_lable)
training_data = list(training_data)
test_data = zip(test_image,test_lable)
test_data = list(test_data)
#         x = a.reshape(28,28)
#         fig = plt.figure()
#         plotwindow = fig.add_subplot(111)
#         plt.imshow(x , cmap='gray')
#         plt.show()

In [3]:
# a = [training_data[:10],training_data[10:20]]
# for x, y in a[1]:
#     print(np.argmax(y))
mini_batches = [training_data[k:k+10] for k in range(0, 50000, 10)]
print(len(mini_batches))
print("1:")
a = mini_batches[0]
for x, y in a:
    print(np.argmax(y))
print("2:")
a = mini_batches[2]
for x, y in a:
    print(np.argmax(y))


5000
1:
5
0
4
1
9
2
1
3
1
4
2:
4
0
9
1
1
2
4
3
2
7


In [5]:
def sigmoid_prime(z):
    return sigmoid(z) * (1-sigmoid(z))
def sigmoid(z):#S型神经元
    return 1.0/(1.0+np.exp(-z))
# def sigmoid(x):
#     x_ravel = x.ravel()  # 将numpy数组展平
#     length = len(x_ravel)
#     y = []
#     for index in range(length):
#         if x_ravel[index] >= 0:
#             y.append(1.0 / (1 + np.exp(-x_ravel[index])))
#         else:
#             y.append(np.exp(x_ravel[index]) / (np.exp(x_ravel[index]) + 1))
#     return np.array(y).reshape(x.shape)
class Network(object):
    def __init__(self,sizes):
        self.num_layers = len(sizes) #sipzes神经网络规模例如输入[10,20,10]就代表第一层10个神经元，第二层20个神经元，第三层10个神经元
        self.sizes = sizes
        np.random.seed(0)
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] #每层的偏移
        self.weights = [np.random.randn(y, x)                    #参数矩阵
                       for x, y in zip(sizes[:-1],sizes[1:])]
    def feedforward(self, a):       #前项计算
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)                          
        return a         
    def SGD(self, training_data, epochs, mini_batch_size, eta,#mini_batch_size每次取样的规模
           test_data = None):#eta 是学习速率，#epochs 迭代次数
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs): 
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}:{1} / {2}". format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                            for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                            for b, nb in zip(self.biases, nabla_b)]
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        delta = self.cost_derivative(activations[-1], y) *\
            sigmoid_prime(zs[-1])#是代码的换行符，无任何作用
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
            
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp 
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)

In [6]:
net = Network([784,30,10])
net.SGD(training_data, 30, 10, 0.5, test_data=test_data)

H:\anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


Epoch 0:2761 / 10000
Epoch 1:4627 / 10000
Epoch 2:5688 / 10000
Epoch 3:5670 / 10000
Epoch 4:5428 / 10000
Epoch 5:5995 / 10000
Epoch 6:5736 / 10000
Epoch 7:6135 / 10000
Epoch 8:4927 / 10000
Epoch 9:6057 / 10000
Epoch 10:6314 / 10000
Epoch 11:6391 / 10000
Epoch 12:6880 / 10000
Epoch 13:6461 / 10000
Epoch 14:6734 / 10000
Epoch 15:6069 / 10000
Epoch 16:6324 / 10000
Epoch 17:6255 / 10000
Epoch 18:6136 / 10000
Epoch 19:5258 / 10000
Epoch 20:4933 / 10000
Epoch 21:6109 / 10000
Epoch 22:6249 / 10000
Epoch 23:5406 / 10000
Epoch 24:5915 / 10000
Epoch 25:6420 / 10000
Epoch 26:6274 / 10000
Epoch 27:5776 / 10000
Epoch 28:5356 / 10000
Epoch 29:5367 / 10000


In [ ]:
#         100*784 w1(784*30) w2(30*10)
#         30 10
#         100*784 * w1(784*30) = z[0]
#         z_s[0](100*30) * w2(30*10) = z[1]
#         z_s[1](100*10)

In [ ]:
def training(self, training_data, mini_batch_size):
        #数据预处理，把成组的数据分开
        pdb.set_trace()
        x = np.zeros((mini_batch_size,784))
        y = np.zeros((mini_batch_size,10))
        for (a, b), i in zip(training_data,range(mini_batch_size)):
            x[i] = a
            y[i] = b
        #Forward pass
        z = [] #中间变量矩阵
        z_s = [] #经过S型激活后的中间变量矩阵
        z_s.append(x)
#         print(np.dot(x,self.weights[0]).shape,self.biases[0].shape)
        a = np.dot(x,self.weights[0])+self.biases[0] #经过第一组参数矩阵
        z.append(a)
        a = self.sigmoid(a)
        z_s.append(a)
        a = np.dot(z_s[1],self.weights[1]) + self.biases[1] #经过第二组参数矩阵
        z.append(a)
        z_s.append(self.sigmoid(a))
        y_pred = z_s[2]
            
        #Loss ,计算loss并输出
        loss = np.square(y_pred - y).sum()
#         1*784 w1(784*30) w2(30*10)
#         30 10
#         1*784 * w1(784*30) = z[0]
#         z_s[0](1*30) * w2(30*10) = z[1](1*10)
#         z_s[1](1*10)
        #Back Forward pass 反向传播
        grad_b = [np.zeros(b.shape) for b in self.biases]
        grad_w = [np.zeros(w.shape) for w in self.weights]
        grad_z = []
        a = (y_pred - y) * self.sigmoid_prime(z[-1]) #复合函数求导法,a是偏导数
        grad_z.append(a)
        grad_b[-1] = a
        grad_w[-1] = np.dot(z_s[-2].T,grad_z[0])
#         print("53行,z[-2]尺寸",len(grad_z[0]),len(grad_z[0][0]))
#         print("54行,self.weights[-1]尺寸",self.weights[-1].shape,)
        a = np.dot(grad_z[0],self.weights[-1].T)*self.sigmoid_prime(z[0]) #复合函数求导法
        grad_z.append(a)
        grad_b[-2] = a
        grad_w[-2] = np.dot(z_s[-3].T,grad_z[1]) 
        return grad_w,grad_b,loss